In [ ]:
import $exclude.`org.slf4j:slf4j-log4j12`, $ivy.`org.slf4j:slf4j-nop:1.7.21` // for cleaner logs
//import $profile.`hadoop-2.6`
import $ivy.`org.apache.spark::spark-sql:2.1.0` // adjust spark version - spark >= 2.0
//import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`
import $ivy.`org.jupyter-scala::spark:0.4.2` // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)

import org.apache.spark._
import org.apache.spark.sql._
import jupyter.spark.session._
import Resolvers._

In [ ]:
import $ivy.`org.vegas-viz::vegas:0.3.9`
import vegas._
import vegas.data.External._

In [ ]:
interp.resolvers() = interp.resolvers() :+ Resolver.Http(
  "isarn project",
  "https://dl.bintray.com/isarn/maven/",
  MavenPattern,
  true
)

In [ ]:
import $ivy.`org.isarnproject::isarn-sketches:0.0.3.rc1` // "org.isarnproject" %% "isarn-sketches" % "0.0.2"
import org.isarnproject.sketches._

In [ ]:
val sparkSession = JupyterSparkSession.builder() // important - call this rather than SparkSession.builder()
  .jupyter() // this method must be called straightaway after builder()
  .master("spark://frclust:7077")
  .appName("notebook")
  .getOrCreate()
val sc = sparkSession.sparkContext

In [ ]:
implicit class TDEnhance(td: TDigest) {
  def toVegasXY: Seq[(Double, Double)] = {
    val q = (0.001 +: (0.05 to 0.95 by 0.05) :+ 0.999).toVector
    q.map(q => (td.cdfInverse(q), q))
  }
}

In [ ]:
val raw = sc.parallelize(Vector.fill(1000000) { scala.util.Random.nextGaussian() }, 10)

In [ ]:
val td = raw.aggregate(TDigest.empty())(
    (td, x) => td + x,
    (td1, td2) => td1 ++ td2
)

In [ ]:
Vegas("CDF")
    .withXY(td.toVegasXY)
    .encodeX("x", Quant)
    .encodeY("y", Quant)
    .mark(Line)
    .show

In [ ]:
val trdd = sc.textFile("/data/tox21/tox21_dense_train.csv")

In [ ]:
trdd.count